In [9]:
import os
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

In [10]:
def extract_hog_features(image, image_size=(128, 64)):
    image_resized = cv2.resize(image, image_size)
    winSize = image_size
    blockSize = (8, 8)
    blockStride = (8, 8)
    cellSize = (4, 4)
    nbins = 9
    hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins)
    hog_features = hog.compute(image_resized)
    return hog_features.flatten()

In [11]:
def load_hog_features_from_folders(base_folder, subfolders, specific_folder, image_size=(128, 64)):
    hog_features = []
    labels = []
    for subject in os.listdir(base_folder):
        subject_path = os.path.join(base_folder, subject)
        if os.path.isdir(subject_path):
            for subfolder in subfolders:
                subfolder_path = os.path.join(subject_path, subfolder)
                if os.path.isdir(subfolder_path):
                    specific_folder_path = os.path.join(subfolder_path, specific_folder)
                    if os.path.isdir(specific_folder_path):
                        for root, _, files in os.walk(specific_folder_path):
                            for img_name in files:
                                img_path = os.path.join(root, img_name)
                                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                                if img is not None:
                                    hog_feat = extract_hog_features(img, image_size)
                                    hog_features.append(hog_feat)
                                    labels.append(subject)
    return np.array(hog_features), np.array(labels)

In [12]:
base_folder = '/home/sazid/Model_Based_GAIT_PROJECT/Pre_Processing/output/GEI_Image/'

In [13]:
train_folders = ['nm-01','nm-02', 'nm-03', 'nm-04', 'nm-05', 'nm-06', 'cl-01', 'cl-02'] 
test_folders = ['bg-01', 'bg-02']  
folders = ['000', '018', '036', '054', '072', '090', '108', '126', '144', '162', '180']

In [14]:
output_dir = '/home/sazid/Model_Based_GAIT_PROJECT/Pre_Processing/output/Excelfile'
os.makedirs(output_dir, exist_ok=True)

In [15]:
results = []

In [16]:
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np

# Assuming load_hog_features_from_folders is defined elsewhere

results = []

for train_folder in folders:
    for test_folder in folders:
        print(f'\nTraining on: {train_folder}, Testing on: {test_folder}')
        
        # Load HOG features
        X_train, y_train = load_hog_features_from_folders(base_folder, train_folders, train_folder)
        X_test, y_test = load_hog_features_from_folders(base_folder, test_folders, test_folder)
        
        # Normalize features
        X_train = X_train / np.max(X_train)
        X_test = X_test / np.max(X_test)

        # Apply LDA for dimensionality reduction
        lda = LDA(n_components=min(len(np.unique(y_train)) - 1, X_train.shape[1]))
        lda.fit(X_train, y_train)
        
        # Transform data using LDA
        X_train_lda = lda.transform(X_train)
        X_test_lda = lda.transform(X_test)
        
        # Get eigenvalues and eigenvectors from LDA
        eigenvalues = lda.explained_variance_ratio_  # These are the eigenvalues
        eigenvectors = lda.scalings_  # These are the eigenvectors

        # Sort the eigenvalues and corresponding eigenvectors in descending order
        sorted_indices = np.argsort(eigenvalues)[::-1]
        sorted_eigenvalues = eigenvalues[sorted_indices]
        sorted_eigenvectors = eigenvectors[:, sorted_indices]

        # Extract the top k relevant features (eigenvectors corresponding to the largest eigenvalues)
        k = len(np.unique(y_train)) - 1  # Number of classes - 1
        relevant_features = sorted_eigenvectors[:, :k]

        # Extract irrelevant features (eigenvectors corresponding to the smallest eigenvalues)
        irrelevant_features = sorted_eigenvectors[:, k:]

        # Now relevant_features contains the most significant features for class discrimination
        # irrelevant_features contains the least significant ones

        # Optionally, print or visualize the relevant and irrelevant features
        print(f"Relevant features shape: {relevant_features.shape}")
        print(f"Irrelevant features shape: {irrelevant_features.shape}")

        # You can visualize or analyze these features further based on your needs.

        # Initialize KNN classifiers for evaluation
        knn_types = [
            {'n_neighbors': 3, 'metric': 'euclidean'},
            {'n_neighbors': 3, 'metric': 'manhattan'},
            {'n_neighbors': 3, 'metric': 'minkowski'},
            {'n_neighbors': 3, 'metric': 'chebyshev'}
        ]
        
        for knn_type in knn_types:
            n_neighbors = knn_type['n_neighbors']
            metric = knn_type['metric']
            
            print(f'Evaluating KNN with n_neighbors={n_neighbors} and metric={metric}')
            
            # Train KNN on the LDA-transformed data
            knn = KNeighborsClassifier(n_neighbors=n_neighbors, metric=metric)
            knn.fit(X_train_lda, y_train)
            
            # Predict and evaluate the model
            y_pred = knn.predict(X_test_lda)
            accuracy = accuracy_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred, average='macro')
            recall = recall_score(y_test, y_pred, average='macro')
            f1 = f1_score(y_test, y_pred, average='macro')
            
            # Store results
            results.append({
                'Train Folder': train_folder,
                'Test Folder': test_folder,
                'n_neighbors': n_neighbors,
                'Metric': metric,
                'Accuracy': accuracy,
                'Precision': precision,
                'Recall': recall,
                'F1-Score': f1
            })

# Results now contain the performance of KNN classifiers for different training and testing folder combinations



Training on: 000, Testing on: 000
Relevant features shape: (4608, 122)
Irrelevant features shape: (4608, 0)
Evaluating KNN with n_neighbors=3 and metric=euclidean
Evaluating KNN with n_neighbors=3 and metric=manhattan
Evaluating KNN with n_neighbors=3 and metric=minkowski
Evaluating KNN with n_neighbors=3 and metric=chebyshev

Training on: 000, Testing on: 018


/home/sazid/Model_Based_GAIT_PROJECT/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sazid/Model_Based_GAIT_PROJECT/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sazid/Model_Based_GAIT_PROJECT/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

Relevant features shape: (4608, 122)
Irrelevant features shape: (4608, 0)
Evaluating KNN with n_neighbors=3 and metric=euclidean
Evaluating KNN with n_neighbors=3 and metric=manhattan
Evaluating KNN with n_neighbors=3 and metric=minkowski
Evaluating KNN with n_neighbors=3 and metric=chebyshev

Training on: 000, Testing on: 036


/home/sazid/Model_Based_GAIT_PROJECT/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sazid/Model_Based_GAIT_PROJECT/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sazid/Model_Based_GAIT_PROJECT/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

Relevant features shape: (4608, 122)
Irrelevant features shape: (4608, 0)
Evaluating KNN with n_neighbors=3 and metric=euclidean
Evaluating KNN with n_neighbors=3 and metric=manhattan
Evaluating KNN with n_neighbors=3 and metric=minkowski
Evaluating KNN with n_neighbors=3 and metric=chebyshev

Training on: 000, Testing on: 054


...

/home/sazid/Model_Based_GAIT_PROJECT/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sazid/Model_Based_GAIT_PROJECT/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sazid/Model_Based_GAIT_PROJECT/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

Relevant features shape: (4608, 123)
Irrelevant features shape: (4608, 0)
Evaluating KNN with n_neighbors=3 and metric=euclidean
Evaluating KNN with n_neighbors=3 and metric=manhattan
Evaluating KNN with n_neighbors=3 and metric=minkowski
Evaluating KNN with n_neighbors=3 and metric=chebyshev

Training on: 180, Testing on: 180


/home/sazid/Model_Based_GAIT_PROJECT/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sazid/Model_Based_GAIT_PROJECT/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sazid/Model_Based_GAIT_PROJECT/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

Relevant features shape: (4608, 123)
Irrelevant features shape: (4608, 0)
Evaluating KNN with n_neighbors=3 and metric=euclidean
Evaluating KNN with n_neighbors=3 and metric=manhattan
Evaluating KNN with n_neighbors=3 and metric=minkowski
Evaluating KNN with n_neighbors=3 and metric=chebyshev


/home/sazid/Model_Based_GAIT_PROJECT/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sazid/Model_Based_GAIT_PROJECT/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/sazid/Model_Based_GAIT_PROJECT/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [17]:
results_df = pd.DataFrame(results)
output_file_path = os.path.join(output_dir, 'mm_bg02_01_test_gei.xlsx')
results_df.to_excel(output_file_path, index=False)
print(f"Results saved to: {output_file_path}")

Results saved to: /home/sazid/Model_Based_GAIT_PROJECT/Pre_Processing/output/Excelfile/mm_bg02_01_test_gei.xlsx


In [18]:
# Assuming `hog_features`, `fisher_features`, and `lda_features` are the feature matrices

print("HOG Features:", hog_features.shape[1])  # Total features from HOG
print("Fisher Features:", fisher_features.shape[1])  # Features after Fisher
print("LDA Features:", lda_features.shape[1])  # Features after LDA


NameError: name 'hog_features' is not defined